In [5]:
f_m_q=[1,4,7,10] #1-й месяц квартала: 
s_m_q=[2,5,8,11] #2-й месяц квартала:
t_m_q=[3,6,9,12] #3-й месяц квартала:

def month_train_sample(month, quart, n=12): #то есть на 12 месяцев назад смотрим
    itog_x, itog_quart, itog_y=[],[],[]
    
    for i in range(n,len(month)):
        forecast_date=month[i:i+1].index[0]#прогнозируемый месяц
        forecast_month=forecast_date.month
        
        if forecast_month in f_m_q:#если данные заканчиваются на  1,4,7,10 месяц
            
            b=pd.concat([month[i-n:i],pd.Series([np.nan,np.nan,np.nan],index=month[i:i+3].index)])
            q=quart.loc[get_lag(forecast_date,10):get_lag(forecast_date,-2)]
            
        elif forecast_month in s_m_q:#2-й месяц квартала 2,5,8,11
            
            b=pd.concat([month[i-n-1:i],pd.Series([np.nan,np.nan],index=month[i:i+2].index)])          
            q=quart.loc[get_lag(forecast_date,12):get_lag(forecast_date,-1)]
            
        else: #3-й месяц квартала 3,6,9,12
            
            b=pd.concat([month[i-n-2:i],pd.Series([np.nan],index=month[i:i+1].index)])   
            q=quart.loc[get_lag(forecast_date,13):forecast_date]
            
        itog_x.append(b)
        itog_quart.append(q)
        itog_y.append(month[i:i+1])
    return itog_x, itog_quart, itog_y

In [4]:
from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta

def get_first_day(dt, d_years=0, d_months=0):
    y, m = dt.year + d_years, dt.month + d_months
    a, m = divmod(m-1, 12)
    return date(y+a, m+1, 1)

def get_last_day(dt):
    return get_first_day(dt,0,1)+timedelta(-1)

def get_lag(date,month_lag):
    last_d=date+-timedelta(1)#возвращаем последнюю дату данных
    with_lag=last_d-relativedelta(months=month_lag) #отматываем на month_lag месяцев назад
    return get_last_day(with_lag)#т.к. отматывается на неск месяцев с даты текущего месяца, то нужно докрутить до последнего числа

In [3]:
f_m_q=[1,4,7,10] #1-й месяц квартала: 
s_m_q=[2,5,8,11] #2-й месяц квартала:
t_m_q=[3,6,9,12] #3-й месяц квартала:

def month_train_sample(month, quart, n=12): #то есть на 12 месяцев назад смотрим
    itog_x, itog_quart, itog_y=[],[],[]
    
    for i in range(n,len(month)):
        forecast_date=month[i:i+1].index[0]#прогнозируемый месяц
        forecast_month=forecast_date.month
        
        if forecast_month in f_m_q:#если данные заканчиваются на  1,4,7,10 месяц
            
            b=pd.concat([month[i-n:i],pd.Series([np.nan,np.nan,np.nan],index=month[i:i+3].index)])
            q=quart.loc[get_lag(forecast_date,10):get_lag(forecast_date,-2)]
            
        elif forecast_month in s_m_q:#2-й месяц квартала 2,5,8,11
            
            b=pd.concat([month[i-n-1:i],pd.Series([np.nan,np.nan],index=month[i:i+2].index)])          
            q=quart.loc[get_lag(forecast_date,12):get_lag(forecast_date,-1)]
            
        else: #3-й месяц квартала 3,6,9,12
            
            b=pd.concat([month[i-n-2:i],pd.Series([np.nan],index=month[i:i+1].index)])   
            q=quart.loc[get_lag(forecast_date,13):forecast_date]
            
        itog_x.append(b)
        itog_quart.append(q)
        itog_y.append(month[i:i+1])
    return itog_x, itog_quart, itog_y

In [ ]:
def adfuller_series(X):
    test = adfuller(X)
    print ('adf: ', test[0])
    print ('p-value: ', test[1])
    print('Critical values: ', test[4])
    if test[0]> test[4]['5%']: 
        print ('есть единичные корни, ряд не стационарен')
    else:
        print ('единичных корней нет, ряд стационарен')

In [ ]:
from datetime import date, timedelta

def get_first_day(dt, d_years=0, d_months=0):
    y, m = dt.year + d_years, dt.month + d_months
    a, m = divmod(m-1, 12)
    return date(y+a, m+1, 1)

def get_last_day(dt):
    return get_first_day(dt,0,1)+timedelta(-1)

def sammary_acc(y_test,y_pred):
    # y_pred=np.array(pred_y_c)
    tp=0
    fp=0
    fn=0
    tn=0
    for i in np.array([y_test,y_pred]).T:
        if i[0]==1 and i[1]==1: tp+=1
        if i[0]==0 and i[1]==1: fp+=1
        if i[0]==1 and i[1]==0: fn+=1
        if i[0]==0 and i[1]==0: tn+=1
    print('','TP =',tp, 'FP =', fp,'\n','FN =', fn,'TN =', tn)
    return tp,fp,fn,tn
def roc_auc_pic(model, y_train, y_test, X_train, X_test):
    y_hat_train = model.predict_proba(X_train)[:,1]
    y_hat_test = model.predict_proba(X_test)[:,1]
    
    fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_hat_train)
    fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_hat_test)
    roc_auc_train = roc_auc_score(y_train, y_hat_train)
    roc_auc_test = roc_auc_score(y_test, y_hat_test)

    matplotlib.rcParams['figure.figsize'] = (8, 8)
    plt.plot(fpr_train, tpr_train, label='Train ROC AUC {0}'.format(roc_auc_train))
    plt.plot(fpr_test, tpr_test, label='Test ROC AUC {0}'.format(roc_auc_test))
    plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6))
    plt.xlim([-0.01, 1])
    plt.ylim([0, 1.01])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.show()
    
import matplotlib.gridspec as gridspec
def print_3plots(history):
    fig2 = plt.figure(figsize=(18, 4))
    spec2 = gridspec.GridSpec(ncols=3, nrows=1, figure=fig2)
    ax1 = fig2.add_subplot(spec2[0, 0])
    ax2 = fig2.add_subplot(spec2[0, 1])
    ax3 = fig2.add_subplot(spec2[0, 2])


    ax1.plot(history.history['loss'])
    ax1.plot(history.history['val_loss'])
    ax1.set_title('model loss')
    ax1.set_ylabel('loss')
    ax1.set_xlabel('epoch')
    ax1.legend(['train', 'test'], loc='best')

    ax2.plot(history.history['acc'])#точность процент правильно классифицированных примеров
    ax2.plot(history.history['val_acc'])
    ax2.set_title('model accuracy')
    ax2.set_ylabel('accuracy')
    ax2.set_xlabel('epoch')
    ax2.legend(['train', 'test'], loc='best')

    ax3.plot(history.history['auc_roc'])#точность процент правильно классифицированных примеров
    ax3.plot(history.history['val_auc_roc'])
    ax3.set_title('auc_roc')
    ax3.set_ylabel('auc_roc')
    ax3.set_xlabel('epoch')
    ax3.legend(['train', 'test'], loc='best')
    # fig.savefig(str(pp)+'foo2.png') 
    print ('Отношение числа наблюдений к кол-ву оцениваемых параметров:{}'.format(round((X_train.shape[0]*X_train.shape[1])/size(model),2)))
    print ()
    b1='last val_loss = {}'.format(round(history.history['val_loss'][-1],3))
    print(b1)
    b2='last accuracy = {}'.format(round(history.history['val_acc'][-1],3))
    print(b2)
    b3='last val_auc_roc = {}'.format(round(history.history['val_auc_roc'][-1],3))
    print(b3)
    print()
    return [b1,b2,b3]

In [ ]:
def print_rez(threshold, dates, y_test, y_probs,for_NN=True):
    #threshold=0.6
    a0='Порог = {}'.format(threshold)
    y_pred_my=[]
    for i in y_probs.T[1]:
        if i>threshold: y_pred_my.append(1) 
        else: y_pred_my.append(0) 
#     np.array(y_pred_my)
    # это что значит?? в лог рег второй ряд вероятностей - это вероятность роста
    a1='На тестовой выборке:\n периодов роста: {} \n периодов рецессии: {}'.format(Counter(y_test)[1],Counter(y_test)[0])
    print(a1)
    print()
    # sammary_acc(y_test,y_pred)# это для проверки расчета конф матрицы другой моей функцией
    print(' H0- рецессия, а H1- рост!!!!')
    print('','FP - ошибка первого рода (говорим, что рост, но на самом деле рецессия)',
          '\n','FN - ошибка второго рода (говорим, что рецессия, но на самом деле рост)')
    print()
    a2=str(Counter(np.array(y_pred_my)*2-y_test))
    print(a2)
    d=Counter(np.array(y_pred_my)*2-y_test)
    tp,fp,fn,tn=d[1],d[2],d[-1],d[0]
    a3='TP:{} FP:{} \nFN:{} TN:{}'.format(d[1],d[2],d[-1],d[0])
    print(a3)
    print()
    pr_0=round(tn/(tn+fn+0.1**10),2)
    rec_0=round(tn/(tn+fp+0.1**10),2)
    a4=''.join(['','precision(TN/TN+FN) for 0 =',str(pr_0),'\n','recall(TN/TN+FP) for 0 =',str(rec_0)])
    print(a4)
    print()
    pr_1=round(tp/(tp+fp+0.1**10),2)
    rec_1=round(tp/(tp+fn+0.1**10),2)
    a5=''.join(['','precision(TP/TP+FP) for 1 =',str(pr_1),'\n','recall(TP/TP+FN) for 1 =',str(rec_1)])
    print(a5)
    print()
    
    if for_NN==True:      
        log_loss_test=skmetrix.log_loss(y_true=np.array(transf_Y(y_test)), y_pred=y_probs)# без порога, потому что кроссэнтропия, там вероятности 
        a6='log_loss_test= {}'.format(round(log_loss_test,3))
        print(a6)
        accuracy_score_test=skmetrix.accuracy_score(y_true=np.array(y_test), y_pred=np.array(y_pred_my))#только здксь участвует порог
        a7='accuracy_score_test= {}'.format(round(accuracy_score_test,3))
        print(a7)  
        roc_auc_score_test=skmetrix.roc_auc_score(y_true=np.array(transf_Y(y_test)), y_score=y_probs)
        a8='roc_auc_score_test= {}'.format(round(roc_auc_score_test,3))
        print(a8)
    else:
        log_loss_test=skmetrix.log_loss(y_true=np.array(y_test), y_pred=y_probs)# без порога, потому что кроссэнтропия, там вероятности 
        a6='log_loss_test= {}'.format(round(log_loss_test,3))
        print(a6)
        accuracy_score_test=skmetrix.accuracy_score(y_true=np.array(y_test), y_pred=np.array(y_pred_my))#только здксь участвует порог
        a7='accuracy_score_test= {}'.format(round(accuracy_score_test,3))
        print(a7)
        roc_auc_score_test=skmetrix.roc_auc_score(y_true=np.array(y_test), y_score=y_probs.T[1])
        a8='roc_auc_score_test= {}'.format(round(roc_auc_score_test,3))
        print(a8)

    plot_test(dates,y_test,y_probs.T[1])
    return [a0,a1,a3,a4,a5,a6,a7,a8]


In [1]:
def plot_test(dates,y_test,y_probs):
    plt.subplots(figsize=(18,4))
    plt.plot(dates,y_test, color='black', label = 'Original data')#проверить правильно ли дата разбивается
    plt.plot(dates,y_probs, color='blue', label = 'Predicted data')
    plt.legend(loc='best')
    plt.show()

In [2]:
from datetime import datetime
def ctm():  
    datetime_NY = datetime.now()   
    return datetime_NY.strftime("%d-%m-%Y %H:%M:%S")

def intellegens(text):
    with open('history_run.txt', 'a') as file:
        file.write(text+ '\n\n')

In [ ]:
def transf_Y(Yipp):#для нейронки когда на выходе логистическая функция активации
    Y=[]
    for i in range(len(Yipp)):
        try:
            if Yipp[i]==1:
                y_i=[1,0]
            else:
                y_i=[0,1]
        except: print('e')
        Y.append(y_i)
    return(Y)

In [6]:
def fplot(t,y1,y2,ipp_bin_sm3,n):
    fig, ax1 = plt.subplots(figsize=(18,6))
    ax1.plot(t,y1, 'b-')
    ax1.set_xlabel('Time')
    ax1.set_ylabel(y1.name, color='b')
    ax1.tick_params('y', colors='b')
    ax1.fill_between(t, min(y1), max(y1), where=ipp_bin_sm3 < 0,
                facecolor='blue', alpha=0.1)
    ax2 = ax1.twinx()
    s2 = y2.fillna(method='pad')
    ax2.plot(t,s2, 'k')
    ax2.set_ylabel(s2.name, color='black')
    ax2.tick_params('y', colors='black')
    fig.tight_layout()
#     plt.show()
    fig.savefig('overlay/dual'+str(n)+ctm()+'.png')